In [2]:
#用于1车道1交通灯和 pytorch测试
#作者lukeliuli@163.com
# -*- coding: utf-8 -*-
"""
@author: lukeliuli@163.com
"""
#%cd /root/myPrjs/SUMONBDT

import numpy as np
import os
import sys

import random
import csv

if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
    


import libsumo

path = os.getcwd()
path2 = path+"\sumoCfgs\my1Lane1TlsVeh3-server.sumocfg"#window
path2 = path+"/sumoCfgs1/my1Lane1TlsVeh3-server.sumocfg"#linux
sumoBinary = "sumo"
libsumo.start([sumoBinary, "-c", path2])


#定义模拟过程中动态可调整参数
params ={}
params["GreenTime"] = 15
params["redTime"] = 25
params["ObjVehPosInXY"] = [-100,0]
params["MaxAcc"] = 1
params["DrvResTime"] = 0.5
params["addedVeh"] = 2+1
params["accel"]=2.8    
params["tau"]=1  
    
numSim = 1000
nextTLSID = "2"
veh = "00003"
counterNoLowSpeed = 0
dataAllSim = []
for i in range(numSim):
    print("##################start sim %d##################" %i)
    str1 = "./trainData/redTLS_"+str(i)+"_sim.csv"
    csv_file = open(str1,"w+")    
    writer = csv.writer(csv_file)   



    libsumo.start([sumoBinary, "-c", path2])
    stepNum = 0
    requireStop = 0
    redLowSpeed = 0
    data = []
    ##修改出发位置，随机参数
    X =  params["ObjVehPosInXY"][0]+min(random.random()*100,80)#注意坐标的原点
    Y =  params["ObjVehPosInXY"][1]
    libsumo.vehicle.moveToXY(veh,"e1to2", 0, X, Y, -1073741824.0, 1)
    
    ##修改加速，反应时间
    vehOthers = "EBUS2"
    accel = params["accel"]
    accel = max(accel*random.random(),1)
    libsumo.vehicletype.setAccel(vehOthers,accel)
    
    tau = params["tau"]
    tau = max(tau*random.random(),0.3)
    libsumo.vehicletype.setTau(vehOthers,tau)
    
    ####修改绿灯时间，随机参数
    """
    greenTime =  params["GreenTime"]#注意坐标的原点
    greenTime = max(random.random()*greenTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    greenDurTime = lgc1.phases[1].duration
    yellowDurTime = lgc1.phases[2].duration
    redDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = greenTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
    """
    
    ####修改逻辑和红灯时间，随机参数
    redTime =  params["redTime"]#注意坐标的原点
    redTime = max(random.random()*redTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    
    lgc1.phases[1].state = 'r'
    redDurTime = lgc1.phases[1].duration
    
    yellowDurTime = lgc1.phases[2].duration
    
    lgc1.phases[3].state = 'G'
    greenDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = redTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
   
    #print(X,greenTime,greenDurTime,yellowDurTime,redDurTime)
    #print(lgc1)
    
    data = []
    
    ##先运行一步
    libsumo.simulationStep()
    stepNum += 1
    while requireStop == 0:
        libsumo.simulationStep()
        stepNum += 1
        
        ###交通灯设置
        ####https://sumo.dlr.de/daily/pydoc/traci._vehicle.html
        nextTLSID = "2"
        TLSPos = 0
        states = libsumo.trafficlight.getRedYellowGreenState(nextTLSID)
        timeT = libsumo.simulation.getCurrentTime()  # 当前时间
        nextSwitch = libsumo.trafficlight.getNextSwitch(nextTLSID)
        phaseName = libsumo.trafficlight.getPhaseName(nextTLSID)
        phase = libsumo.trafficlight.getPhase(nextTLSID)
        phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
        curTime = timeT/1000
        phaseLeftTime = nextSwitch - curTime
        
        #车设置
        veh = "00003"
        tlsList = libsumo.vehicle.getNextTLS(veh)
        #tlsID_T, tlsIndex, dist, state = tlsList[0]
       
        
        vehPos = libsumo.vehicle.getPosition(veh)[0]
        speed = libsumo.vehicle.getSpeed(veh)
        vehMaxSpeed = libsumo.vehicle.getMaxSpeed(veh)

        laneID = libsumo.vehicle.getLaneID(veh)
        #meanSpeed = libsumo.lane.getLastStepMeanSpeed(laneID)
        laneMaxSpeed = libsumo.lane.getMaxSpeed(laneID)

        vehMaxSpeed = min(vehMaxSpeed, laneMaxSpeed)
        
        dist = TLSPos-vehPos
        arrivalTime1 = min(100,dist / (speed + 0.001))
        arrivalTime2 = min(100,dist / vehMaxSpeed)
        #arrivalTime3 = min(100,dist / (meanSpeed+0.001))

        #predictMeanSpeed1min = laneMaxSpeed  #1分钟前绿灯时,当前场景下目标车道的平均速度
        #arrivalTime4 = min(dist / predictMeanSpeed1min,100)
        
        ##

       
        
        if states == "r" and  speed<10/3.6:
            print("low speed")
            redLowSpeed = 1
            requireStop = 1
            
        if states == "G" and  speed<10/3.6:
            print("low speed")
            redLowSpeed= 1
            requireStop = 1

      
            
            
        if timeT/1000 > 30 or dist<=0:  # if time is over 30 second stoping the simulation
            requireStop = 1
        #name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]    
        laneAVGSpeed = 40/3.6
        data1 =[phaseLeftTime,dist,speed,40/3.6,dist/(speed+0.01),dist/laneAVGSpeed]
        
        #前面只有停止的车，不存在运动的车,没有的车位置为0
        vehTmp = "00001"
        vehPos1 = libsumo.vehicle.getPosition(vehTmp)[0]
        vehTmp = "00002"
        vehPos2 = libsumo.vehicle.getPosition(vehTmp)[0]
        #name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","速度一直高于1m/s"]    
        laneAVGSpeed = 60/3.6
        data2 =[TLSPos-vehPos1,TLSPos-vehPos2,0,0,0,0,0,0]
        #print("#####tls state: %s,duration:%.3f,running time(ms):%d,passTLS:%d,dist:%d,speed:%.2f" %(states,phaseDur,timeT,passTLSNoDeAcc,dist,speed))
        data.append(data1+data2)
        
       
    
    libsumo.close()
    print("###############################sim %d close" %i)
    name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
    name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","红灯前低速甚至停止"]    
    nameTmp = name1+name2
    writer.writerow(nameTmp)
    #print(len(data))
    for i in range(len(data)):
        rowTmp=data[i]
        #print(type(rowTmp))
        rowTmp.extend([redLowSpeed])
        #print(rowTmp)
        writer.writerow(rowTmp)
        dataAllSim.append(rowTmp)
    csv_file.close()
    
    if redLowSpeed == 0:
            print("########################################redLowSpeed == 0")
            print("########################################redLowSpeed == 0")
            print("########################################redLowSpeed == 0")
            print("########################################redLowSpeed == 0")
            counterNoLowSpeed= counterNoLowSpeed+1
print("finish numSim:%d,LowSpeed:%d" %(numSim,numSim-counterNoLowSpeed))

#########################################################################################################
name1 = ["当前红灯剩余时间","距离红灯的位置","当前速度","车道预计平均速度","预计到达时间1","预计到达时间2"]   
name2 = ["红灯前车辆1位置","车辆2位置","车辆3位置","车辆4位置","车辆5位置","车辆6位置","车辆7位置","车辆8位置","红灯前低速甚至停止"] 
name1 = ["redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
name2 = ["vehPos_1","vehPos_2","vehPos_3","vehPos_4","vehPos_5","vehPos_6","vehPos_7","vehPos_8","lowSpeedFlag"] 
headers = name1+name2
str1 = "./trainData/dataAllsim.csv"
with open(str1,'w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(dataAllSim)
#############################################################################################################    
%rm -rf ./trainData/redTLS*.csv
%zip -q  ./trainData/dataAllSim.zip /root/myPrjs/SUMONBDT/trainData/dataAllsim.csv

Loading configuration ... done.
##################start sim 0##################
Loading configuration ... done.
low speed
###############################sim 0 close
##################start sim 1##################
Loading configuration ... done.
low speed
###############################sim 1 close
##################start sim 2##################
Loading configuration ... done.
low speed
###############################sim 2 close
##################start sim 3##################
Loading configuration ... done.
low speed
###############################sim 3 close
##################start sim 4##################


Loading configuration ... done.
low speed
###############################sim 4 close
##################start sim 5##################
Loading configuration ... done.
low speed
###############################sim 5 close
##################start sim 6##################
Loading configuration ... done.
low speed
###############################sim 6 close
##################start sim 7##################
Loading configuration ... done.
low speed
###############################sim 7 close
##################start sim 8##################
Loading configuration ... done.
low speed
###############################sim 8 close
##################start sim 9##################
Loading configuration ... done.
low speed
###############################sim 9 close
##################start sim 10##################
Loading configuration ... done.
low speed
###############################sim 10 close
##################start sim 11##################
Loading configuration ... done.
low speed
########################

Loading configuration ... done.
low speed
###############################sim 21 close
##################start sim 22##################
Loading configuration ... done.
low speed
###############################sim 22 close
##################start sim 23##################
Loading configuration ... done.
low speed
###############################sim 23 close
##################start sim 24##################
Loading configuration ... done.
low speed
###############################sim 24 close
##################start sim 25##################
Loading configuration ... done.
low speed
###############################sim 25 close
##################start sim 26##################
Loading configuration ... done.
low speed
###############################sim 26 close
##################start sim 27##################
Loading configuration ... done.
low speed
###############################sim 27 close
##################start sim 28##################
Loading configuration ... done.
low speed
#############

###############################sim 34 close
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
##################start sim 35##################
Loading configuration ... done.
low speed
###############################sim 35 close
##################start sim 36##################
Loading configuration ... done.
low speed
###############################sim 36 close
##################start sim 37##################
Loading configuration ... done.
low speed
###############################sim 37 close
##################start sim 38##################
Loading configuration ... done.
low speed
###############################sim 38 close
##################start sim 39##################


Loading configuration ... done.
low speed
###############################sim 39 close
##################start sim 40##################
Loading configuration ... done.
low speed
###############################sim 40 close
##################start sim 41##################
Loading configuration ... done.
low speed
###############################sim 41 close
##################start sim 42##################
Loading configuration ... done.
low speed
###############################sim 42 close
##################start sim 43##################
Loading configuration ... done.
low speed
###############################sim 43 close
##################start sim 44##################
Loading configuration ... done.


...

low speed
###############################sim 970 close
##################start sim 971##################
Loading configuration ... done.
###############################sim 971 close
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
##################start sim 972##################
Loading configuration ... done.
low speed
###############################sim 972 close
##################start sim 973##################
Loading configuration ... done.
###############################sim 973 close
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
##################start sim 974##################
Loading configuration ... done.
low speed
######

Loading configuration ... done.
low speed
###############################sim 975 close
##################start sim 976##################
Loading configuration ... done.
low speed
###############################sim 976 close
##################start sim 977##################
Loading configuration ... done.
low speed
###############################sim 977 close
##################start sim 978##################
Loading configuration ... done.
low speed
###############################sim 978 close
##################start sim 979##################
Loading configuration ... done.
###############################sim 979 close
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
##################start sim 980##################
Loading configuration ... done.
###############################sim 980 close
####################

Loading configuration ... done.
low speed
###############################sim 988 close
##################start sim 989##################
Loading configuration ... done.
low speed
###############################sim 989 close
##################start sim 990##################
Loading configuration ... done.
low speed
###############################sim 990 close
##################start sim 991##################
Loading configuration ... done.
low speed
###############################sim 991 close
##################start sim 992##################
Loading configuration ... done.
low speed
###############################sim 992 close
##################start sim 993##################
Loading configuration ... done.


###############################sim 993 close
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
########################################redLowSpeed == 0
##################start sim 994##################
Loading configuration ... done.
low speed
###############################sim 994 close
##################start sim 995##################
Loading configuration ... done.
low speed
###############################sim 995 close
##################start sim 996##################
Loading configuration ... done.
low speed
###############################sim 996 close
##################start sim 997##################
Loading configuration ... done.
low speed
###############################sim 997 close
##################start sim 998##################
Loading configuration ... done.
low speed
###############################sim 998 close
##################start sim 999###########

UsageError: Line magic function `%zip` not found.


In [3]:
!zip -q  ./trainData/dataAllSim.zip /root/myPrjs/SUMONBDT/trainData/dataAllsim.csv

In [ ]:
!git add .
